To implement the BERT for Dialect Arabic I choose to use google prebuild model.

In [ ]:
#Mohammed Alom R00144214

#Importing required modules
import torch
from tqdm.notebook import tqdm
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
#Checking how many gpu is available from colab to run the program.
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from google.colab import drive
drive.mount('/content/gdrive')
#Checking where the model will save
!ls
get_ipython().system('ls')

     |████████████████████████████████| 1.4MB 8.0MB/s 
     |████████████████████████████████| 2.9MB 48.0MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9376d9c0f3299891c8d60f4dd79453246d18a77cd43fc74d4edceca1483a2383
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Number of GPUs Available:  1
Mounted at /content/gdrive
gdrive	sample_data
gdrive	sample_data


In [ ]:
#Loading the file form google drive
train_file = '/content/gdrive/My Drive/Colab Notebooks/nlp-data/MADAR-Corpus-26-train.tsv'
#train_file = '/content/gdrive/My Drive/Colab Notebooks/nlp-data/MADAR-Corpus-6-train.tsv'
df = pd.read_csv(train_file, sep='\t')

In [ ]:
df.head()

,هناك ، أمام بيانات السائح تماما .,MSA
0,لم اسمع بهذا العنوان من قبل بالقرب من هنا .,MSA
1,استمر في السير في هذا الطريق حتى تجد صيدلية .,MSA
2,كم تكلفة الإفطار ؟,MSA
3,كيف أستطيع مساعدتك ؟,MSA
4,اتجه يسارا عند الناصية الثالثة .,MSA


In [ ]:
#Total each categories levels
df['MSA'].value_counts()

DAM    1600
ASW    1600
RAB    1600
TUN    1600
ALE    1600
MUS    1600
AMM    1600
TRI    1600
MOS    1600
BAS    1600
BEI    1600
FES    1600
JED    1600
BAG    1600
KHA    1600
ALX    1600
SAN    1600
SAL    1600
ALG    1600
SFX    1600
CAI    1600
RIY    1600
BEN    1600
DOH    1600
JER    1600
MSA    1599
Name: MSA, dtype: int64

In [ ]:
#Calculating the unique categories
label_categories = df.MSA.unique()
label_dict = {}
for index, label_categories in enumerate(label_categories):
  label_dict[label_categories] = index
label_dict

{'ALE': 16,
 'ALG': 7,
 'ALX': 6,
 'AMM': 8,
 'ASW': 9,
 'BAG': 17,
 'BAS': 18,
 'BEI': 1,
 'BEN': 19,
 'CAI': 2,
 'DAM': 10,
 'DOH': 3,
 'FES': 20,
 'JED': 11,
 'JER': 12,
 'KHA': 21,
 'MOS': 22,
 'MSA': 0,
 'MUS': 23,
 'RAB': 4,
 'RIY': 13,
 'SAL': 24,
 'SAN': 14,
 'SFX': 15,
 'TRI': 25,
 'TUN': 5}

In [ ]:
df['label'] = df.MSA.replace(label_dict)

In [ ]:
df.head()

,هناك ، أمام بيانات السائح تماما .,MSA,label
0,لم اسمع بهذا العنوان من قبل بالقرب من هنا .,MSA,0
1,استمر في السير في هذا الطريق حتى تجد صيدلية .,MSA,0
2,كم تكلفة الإفطار ؟,MSA,0
3,كيف أستطيع مساعدتك ؟,MSA,0
4,اتجه يسارا عند الناصية الثالثة .,MSA,0


In [ ]:
#Sliting the train and test
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['MSA', 'label', 'data_type']).count()

هناك ، أمام بيانات السائح تماما .
MSA label data_type                                   
ALE 16    train                                   1360
          val                                      240
ALG 7     train                                   1360
          val                                      240
ALX 6     train                                   1360
          val                                      240
AMM 8     train                                   1360
          val                                      240
ASW 9     train                                   1360
          val                                      240
BAG 17    train                                   1360
          val                                      240
BAS 18    train                                   1360
          val                                      240
BEI 1     train                                   1360
          val                                      240
BEN 19    train                                   1360
          val                                      240
CAI 2     train                                   1360
          val                                      240
DAM 10    train                                   1360
          val                                      240
DOH 3     train                                   1360
          val                                      240
FES 20    train                                   1360
          val                                      240
JED 11    train                                   1360
          val                                      240
JER 12    train                                   1360
          val                                      240
KHA 21    train                                   1360
          val                                      240
MOS 22    train                                   1360
          val                                      240
MSA 0     train                                   1359
          val                                      240
MUS 23    train                                   1360
          val                                      240
RAB 4     train                                   1360
          val                                      240
RIY 13    train                                   1360
          val                                      240
SAL 24    train                                   1360
          val                                      240
SAN 14    train                                   1360
          val                                      240
SFX 15    train                                   1360
          val                                      240
TRI 25    train                                   1360
          val                                      240
TUN 5     train                                   1360
          val                                      240

In [ ]:
#Loading pretrained google model converting all to lower case.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# encoding and hidden layer weights will generate a dictionary which will contains the input_ids, token_type_ids, 
#add_special_tokens, attention_mask as list for each input sentence.

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].MSA.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].MSA.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
#Train and Test data length
len(dataset_train), len(dataset_val)

(35359, 6240)

In [ ]:
#Loading the pretaind google - bert-base-uncased model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#Loading train and test dataset
batch_size = 3
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
#Optimising the model parameters for neurl networks
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
#Here epoch can be changed I set it to 5.
#hyperparameters set to 5 that the number of times the learning algorithm with run
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
#Function for to test accuracy and the F1 score the best value is 1 and worst value is 0.
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

#Function for getting each class accuracy separately
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
#Seed value can be changed here I have set it to 15.
#it will generate the same random number every time with the same seed value
seed_val = 15
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
#Selecting the device here am using the GPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
#Evaluting the model to calculate the loss, prediction, and true value
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    for batch in dataloader_val: 
        batch = tuple(b.to(device) for b in batch)  
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        #finding the prediction of the model
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
#Progres and running of the models
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model(**inputs)       
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
           
    #Saving the model to google drive
    torch.save(model.state_dict(),f'/content/gdrive/My Drive/Colab Notebooks/nlp-data/finetuned_BERT_epoch_{epoch}.model')   
    tqdm.write(f'\nEpoch {epoch}')
    #!ls
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    #Calling the evaluate function
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    #Calling the F1 score function
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.10417646509908741
Validation loss: 3.501023191067641e-05
F1 Score (Weighted): 1.0



Epoch 2
Training loss: 9.491669565740898e-06
Validation loss: 1.4671911029696415e-07
F1 Score (Weighted): 1.0



Epoch 3
Training loss: 4.5309032588067654e-08
Validation loss: 0.0
F1 Score (Weighted): 1.0



Epoch 4
Training loss: 1.4934450953511032e-09
Validation loss: 0.0
F1 Score (Weighted): 1.0



Epoch 5
Training loss: 8.73142843511151e-10
Validation loss: 0.0
F1 Score (Weighted): 1.0



In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
#Loading the model form google drive
model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/nlp-data/finetuned_BERT_epoch_1.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
#Calling the evaluate function to prediction and true value
_, predictions, true_vals = evaluate(dataloader_validation)


In [ ]:
#Calculating the accuracy of the each level.
accuracy_per_class(predictions, true_vals)

Class: MSA
Accuracy: 240/240

Class: BEI
Accuracy: 240/240

Class: CAI
Accuracy: 240/240

Class: DOH
Accuracy: 240/240

Class: RAB
Accuracy: 240/240

Class: TUN
Accuracy: 240/240

Class: ALX
Accuracy: 240/240

Class: ALG
Accuracy: 240/240

Class: AMM
Accuracy: 240/240

Class: ASW
Accuracy: 240/240

Class: DAM
Accuracy: 240/240

Class: JED
Accuracy: 240/240

Class: JER
Accuracy: 240/240

Class: RIY
Accuracy: 240/240

Class: SAN
Accuracy: 240/240

Class: SFX
Accuracy: 240/240

Class: ALE
Accuracy: 240/240

Class: BAG
Accuracy: 240/240

Class: BAS
Accuracy: 240/240

Class: BEN
Accuracy: 240/240

Class: FES
Accuracy: 240/240

Class: KHA
Accuracy: 240/240

Class: MOS
Accuracy: 240/240

Class: MUS
Accuracy: 240/240

Class: SAL
Accuracy: 240/240

Class: TRI
Accuracy: 240/240

